<a href="https://colab.research.google.com/github/Lomzem/LomzemStockQuant/blob/main/LomzemStocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import os
import pandas as pd
import datetime
import sys

wanted_sheet_path = '/content/drive/MyDrive/LomzemStockCharts/stocks_to_obtain.xlsx'
raw_stock_data_path = '/content/drive/MyDrive/LomzemStockCharts/rawstockdata'

def get_stock_data(date, ticker):
	api_url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=50000&apiKey=IyEWxobQmnBJh0QwQeNOWH7QggrJGFjp'
	try:
		data = requests.get(api_url).json()
		stock_df = pd.DataFrame(data['results'])
		stock_df.rename(columns = {'c': 'Close', 'h': 'High', 'l': 'Low', 'o': 'Open', 'v': 'Volume', 't': 'Time'}, inplace=True)
		stock_df.insert(0, 'Ticker', ticker)
		stock_df.insert(1, 'FTime', stock_df.apply(lambda row: pd.to_datetime(row['Time']-1.44e+7, unit='ms'), axis=1))
		stock_df.to_excel(f'{raw_stock_data_path}/{date}-{ticker}.xlsx', index=False)
		print(f'Got {raw_stock_data_path}/{date}-{ticker}.xlsx')
	except KeyError:
		print('Wait a little longer')
		print(date, ticker)
		sys.exit()

def read_wanted_sheet():
	df = pd.read_excel(wanted_sheet_path)
	for index, row in df.iterrows():
		files = [file for file in os.listdir(raw_stock_data_path)]
		if not (f'{row["Date"].date()}-{row["Ticker"]}.xlsx') in files:
			if row['Date'].date() < datetime.date.today()-datetime.timedelta(days=1):
				get_stock_data(row['Date'].date(), row['Ticker'].upper())

read_wanted_sheet()

In [2]:
# import pandas as pd
import os

cum_stock_data = pd.DataFrame()

for file in os.listdir('/content/drive/MyDrive/LomzemStockCharts/rawstockdata'):
	df = pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{file}')
	cum_stock_data = pd.concat([cum_stock_data, df])

print(cum_stock_data)
cum_stock_data.to_excel('/content/drive/MyDrive/LomzemStockCharts/cumulative_stock_data.xlsx', index=False)

   Ticker               FTime   Volume      vw    Open   Close    High  \
0    VERU 2022-04-11 06:30:00   7448.0  5.1801  4.9100  5.4700  5.4800   
1    VERU 2022-04-11 06:31:00   6869.0  5.3946  5.2000  5.4500  5.9300   
2    VERU 2022-04-11 06:32:00  49308.0  6.0368  5.4200  5.7200  6.5300   
3    VERU 2022-04-11 06:33:00  38057.0  5.6815  5.7300  5.7100  5.9000   
4    VERU 2022-04-11 06:34:00  29283.0  5.6577  5.7500  5.7000  5.8300   
..    ...                 ...      ...     ...     ...     ...     ...   
16   BEAT 2022-04-19 13:57:00    100.0  1.6749  1.6749  1.6749  1.6749   
17   BEAT 2022-04-19 14:06:00   1143.0  1.6983  1.6983  1.6983  1.6983   
18   BEAT 2022-04-19 14:12:00    100.0  1.6636  1.6636  1.6636  1.6636   
19   BEAT 2022-04-19 14:53:00    150.0  1.6461  1.6461  1.6461  1.6461   
20   BEAT 2022-04-19 15:59:00    408.0  1.6425  1.6500  1.6400  1.6500   

       Low           Time    n       a    op  
0   4.5500  1649673000000   55     NaN   NaN  
1   5.1600  16496

In [9]:
import requests
import os
import pandas as pd
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_excel('/content/drive/MyDrive/LomzemStockCharts/cumulative_stock_data.xlsx')
ddf = pd.DataFrame()

ddf = pd.concat([ddf, df[['Ticker', 'FTime']]])
ddf['FTime'] = ddf['FTime'].dt.date

# df.loc[192]
# Get Open
t_df = df.loc[df['FTime'].dt.time==datetime.time(9,30), ['Ticker', 'FTime', 'Open']]
t_df['FTime'] = t_df['FTime'].dt.date
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Close
t_df = df.loc[df['FTime'].dt.time==datetime.time(15,59), ['Ticker', 'FTime', 'Close']]
t_df['FTime'] = t_df['FTime'].dt.date
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get High
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(9,30)) & (df['FTime'].dt.time<=datetime.time(15,59))]
t_df = t_df[['Ticker', 'FTime', 'High']]
t_df['FTime'] = t_df['FTime'].dt.date
idx = t_df.groupby(['Ticker', 'FTime'])['High'].transform(max) == t_df['High']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Low
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(9,30)) & (df['FTime'].dt.time<=datetime.time(15,59))]
t_df = t_df[['Ticker', 'FTime', 'Low']]
t_df['FTime'] = t_df['FTime'].dt.date
idx = t_df.groupby(['Ticker', 'FTime'])['Low'].transform(min) == t_df['Low']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Prev Close
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['PrevClose'] = ddf.groupby('Ticker').shift(1)['Close']

# Get Volume
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,0)) & (df['FTime'].dt.time<=datetime.time(20,0))]
t_df = t_df[['Ticker', 'FTime', 'Volume']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df = t_df.groupby(['Ticker', 'FTime'])['Volume'].sum()
t_df.drop_duplicates(inplace=True)
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Premarket Volume
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,0)) & (df['FTime'].dt.time<datetime.time(9,30))]
t_df = t_df[['Ticker', 'FTime', 'Volume']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df.rename(columns={'Volume': 'PVolume'}, inplace=True)
t_df = t_df.groupby(['Ticker', 'FTime'])['PVolume'].sum()
t_df.drop_duplicates(inplace=True)
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Gap %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['Gap%'] = (ddf['Open'] / ddf['PrevClose'] - 1) * 100

# Get Premarket High
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,00)) & (df['FTime'].dt.time<datetime.time(9,30))]
t_df = t_df[['Ticker', 'FTime', 'High']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df.rename(columns={'High': 'PHigh'}, inplace=True)
idx = t_df.groupby(['Ticker', 'FTime'])['PHigh'].transform(max) == t_df['PHigh']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])
ddf.insert(5, 'PHigh', ddf.pop('PHigh'))

# Get Change From Open %

ddf['ChangeFromOpen%'] = (ddf['Close']/ddf['Open']-1)*100

# Get HOD - PrevClose %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['HOD-PrevClose%'] = (ddf['High'] / ddf['PrevClose'] - 1) * 100

# Get HOD - PrevHOD %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['HOD-PrevHOD%'] = (ddf['High'] / ddf.groupby('Ticker').shift(1)['High'] - 1) * 100

# Get Volume Ratio
ddf['Vol Ratio'] = ddf['Volume']/ddf['PVolume']

ddf.rename(columns={'FTime': 'Date'}, inplace=True)

ddf['closedRed'] = ddf['Close'] < ddf['Open']

### PATTERN: Overextended Gap Down

def overextended_gap_down():
  ddf['isOEGD'] = False
  prev_change_from_open = ddf.groupby('Ticker').shift(1)['ChangeFromOpen%']
  ddf.loc[(prev_change_from_open > 30) & (ddf['Gap%']<0), ['isOEGD']] = True

### PATTERN: Gap Up Short

def gap_up_short():
  ddf['isGUS'] = False
  ddf.loc[(ddf['Gap%'] > 35) & (ddf['closedRed']==True) & (ddf['Low'] / ddf['Open']-1 <= -0.1), ['isGUS']] = True

overextended_gap_down()
gap_up_short()

ddf
# print(ddf.head())
# print(ddf.groupby('Ticker')['Close'].shift(1).head())
# t_df

,Ticker,Date,Open,Close,High,PHigh,Low,PrevClose,Volume,PVolume,Gap%,ChangeFromOpen%,HOD-PrevClose%,HOD-PrevHOD%,Vol Ratio,closedRed,isOEGD,isGUS
0,ADGI,2022-03-29,3.8400,3.8500,4.0300,3.9500,3.8200,NaN,1.403665e+06,7.888000e+03,NaN,0.260417,NaN,NaN,177.949417,False,False,False
1,ADGI,2022-03-30,5.7300,5.0400,6.1400,7.1100,5.0000,3.8500,9.205632e+07,1.839324e+07,48.831169,-12.041885,59.480519,52.357320,5.004900,True,False,True
2,ADGI,2022-03-31,4.9850,4.5600,4.9900,5.7000,4.5250,5.0400,8.306119e+06,1.052830e+06,-1.091270,-8.525577,-0.992063,-18.729642,7.889326,True,False,False
3,ADN,2022-04-05,2.2200,2.0100,2.2800,2.2600,1.9850,NaN,1.249714e+06,2.330000e+03,NaN,-9.459459,NaN,NaN,536.357940,True,False,False
4,ADN,2022-04-06,2.5800,3.6350,3.9800,2.6600,2.4700,2.0100,1.621037e+08,1.049987e+06,28.358209,40.891473,98.009950,74.561404,154.386402,False,False,False
5,ADN,2022-04-07,3.3200,3.3000,3.4500,3.8800,3.0400,3.6350,1.916083e+07,2.447205e+06,-8.665750,-0.602410,-5.089409,-13.316583,7.829678,True,True,False
6,ADN,2022-04-08,3.3600,2.9401,3.5500,3.3900,2.8000,3.3000,9.323953e+06,4.042720e+05,1.818182,-12.497024,7.575758,2.898551,23.063564,True,False,False
7,ADN,2022-04-11,2.8100,2.6800,2.8300,2.9200,2.5700,2.9401,2.888275e+06,1.318660e+05,-4.425020,-4.626335,-3.744771,-20.281690,21.903106,True,False,False
8,AXSM,2022-04-19,40.2600,40.9200,41.6100,46.8000,38.0500,NaN,1.104517e+07,2.037628e+06,NaN,1.639344,NaN,NaN,5.420600,False,False,False
9,BBAI,2022-04-05,8.7900,10.3950,11.2700,8.3200,8.7900,NaN,2.093598e+06,1.250000e+02,NaN,18.259386,NaN,NaN,16748.784000,False,False,False


In [ ]:
import os
import pandas as pd

def search_for_duplicates_in_wanted():
    wanted_sheet_path = '/content/drive/MyDrive/LomzemStockCharts/stocks_to_obtain.xlsx'
    raw_stock_data_path = '/content/drive/MyDrive/LomzemStockCharts/rawstockdata'
    df = pd.read_excel(wanted_sheet_path)
    df.drop_duplicates(inplace = True)
    df.dropna(inplace=True)
    df.sort_values(['Ticker'])
    df['Date'] = df['Date'].dt.date
    df.to_excel(wanted_sheet_path, index = False)

search_for_duplicates_in_wanted()

In [13]:
#PLOTLY TEST

import pandas as pd
import plotly.graph_objects as go
import datetime

def make_plotly_graph(date, ticker):
    # cdf = pd.DataFrame()
    # files = [file for file in os.listdir('/content/drive/MyDrive/LomzemStockCharts/rawstockdata')]
    # for file in files:
    #     # print(file)
    #     if file.endswith(f'{date}-{ticker}.xlsx'):
    #         file_index = files.index(file)
    #         df = pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{files[file_index]}')
    #         cdf = pd.concat([cdf, df])
    #         df = pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{files[file_index-1]}')
    #         cdf = pd.concat([cdf, df])
    
    cdf = pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{date}-{ticker}.xlsx')

    fig = go.Figure(data = [go.Candlestick(x=cdf['FTime'],
                            open = cdf['Open'], high = cdf['High'], close=cdf['Close'], low=cdf['Low']
    )])
    fig.update_layout(xaxis_rangeslider_visible=False, template='plotly_dark', title=f'{ticker} on {date} (1 min)')

    # start_omit_time = datetime.datetime.combine(datetime.date.fromisoformat('-'.join(files[file_index-1].split('-')[0:3])), datetime.time(20,1))
    # end_omit_time = datetime.datetime.combine(date, datetime.time(3,59))

    # fig.update_xaxes(rangebreaks=[dict(bounds=[start_omit_time, end_omit_time])])
    fig.add_vrect(
        x0 = cdf.iloc[0]['FTime'], x1 = datetime.datetime.combine(date, datetime.time(9,29)), fillcolor='#3f3e3f', opacity=0.44, line_width=0
    )
    fig.add_vrect(
        x0 = datetime.datetime.combine(date, datetime.time(16,0)), x1 = cdf.iloc[-1]['FTime'], fillcolor='#3f3e3f', opacity=0.44, line_width=0
    )
    fig.add_hline(y=(ddf.iloc[ddf.index[(ddf['Date']==date) & (ddf['Ticker']==ticker)]-1]['Close']).values[0], line_color='#F6FEAA', line_dash='dot')
    fig.show()

for index, row in ddf.sort_values('Date').iterrows():
    if row['PHigh']/row['PrevClose']-1 > .55:
        make_plotly_graph(row['Date'], row['Ticker'])